In [29]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [43]:

import train
import evaluate
import data_util.data
import data_util.batcher
import data_util.config


importlib.reload(train)
importlib.reload(evaluate)
importlib.reload(data_util.config)
importlib.reload(data_util.data)
importlib.reload(data_util.batcher)

from train import *
from evaluate import *
from data_util.data import *
from data_util.batcher import *


INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000


In [75]:
# load real data
data_path = '/Users/rowancassius/Desktop/capstone/data/context_task_data.tsv'
dat = pd.read_csv(data_path, sep='\t')

# test/train split
x_train, x_test, y_train, y_test = train_test_split(
    dat.TaskSentence.values, 
    dat.Summary.values,
    test_size=0.2,
    random_state=0,
    shuffle=True
)

# fit tokenizer
tokenizer = Tokenizer(lower=True, filters='!"#$%&()*+,./:;<=>?[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(list(x_train)+list(y_train))

# lowercase the data
x_train = tokenizer.sequences_to_texts(tokenizer.texts_to_sequences(x_train))
y_train = tokenizer.sequences_to_texts(tokenizer.texts_to_sequences(y_train))

examples = list(zip(x_train, y_train))
# examples

In [78]:
# Make Vocab
vocab = data.Vocab(
    words = tokenizer.word_index.keys(), 
    max_size=len(tokenizer.word_index))

# Make Batcher
batcher = Batcher(
    examples=examples,
    vocab=vocab, 
    mode='train', 
    batch_size=32, 
    single_pass=False)

max_size of vocab was specified as 2829; we now have 2829 words. Stopping reading.
Finished constructing vocabulary of 2829 total words. Last word added: foundation
INFO:tensorflow:Bucket queue size: 7, Input queue size: 137


In [79]:
random.seed(123)
T.manual_seed(123)
if T.cuda.is_available():
    T.cuda.manual_seed_all(123)

In [80]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [81]:
train_mle = "yes"
train_rl = "no"
mle_weight = 1.0
load_model = None
new_lr = None
rl_weight = 1 - mle_weight

opt = Namespace(train_mle = train_mle, 
                train_rl = train_rl, 
                mle_weight = mle_weight, 
                load_model = load_model,
                new_lr = new_lr, 
                rl_weight = rl_weight)


train_processor = Train(vocab, batcher, opt)

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000


In [82]:
train_processor.trainIters()

0
1
2
3
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
4
iter: 5 mle_loss: 6.479 reward: 0.0000
5
6
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
7
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
8
9
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
iter: 10 mle_loss: 6.066 reward: 0.0000
10
11
12
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
13
14
iter: 15 mle_loss: 4.739 reward: 0.0000
15
16
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
17
18
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
19
iter: 20 mle_loss: 3.955 reward: 0.0000
20
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
21
22
23
24
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
iter: 25 mle_loss: 3.863 reward: 0.0000
25
26
27
28
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
29
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
iter: 

243
244
iter: 245 mle_loss: 2.327 reward: 0.0000
245
246
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
247
248
249
iter: 250 mle_loss: 1.973 reward: 0.0000
model saved at: 
 data/saved_models/0000250.tar
250
251
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
252
253
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
254
iter: 255 mle_loss: 2.020 reward: 0.0000
255
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
256
257
258
259
iter: 260 mle_loss: 1.976 reward: 0.0000
260
261
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
262
263
264
iter: 265 mle_loss: 2.052 reward: 0.0000
265
266
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
267
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
268
269
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
iter: 270 mle_loss: 2.136 reward: 0.0000
270
271
272
273
274
INFO:tensorflow:Bucket queue size: 1000, Input queue size:

iter: 495 mle_loss: 1.375 reward: 0.0000
495
496
497
498
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
499
iter: 500 mle_loss: 1.329 reward: 0.0000
model saved at: 
 data/saved_models/0000500.tar
500
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


In [1]:
task = "validate"
load_model = os.path.join(config.log_root, "data/saved_models/0000500.tar")

opt = Namespace(task = task, load_model = load_model)

# new batcher for evaluation
batcher = Batcher(
    examples=examples[:20],
    vocab=vocab, 
    mode='train', 
    batch_size=10, 
    single_pass=True)

eval_processor = Evaluate(vocab, batcher, opt) 

NameError: name 'os' is not defined

In [85]:
decoded_sents, ref_sents, scores = eval_processor.evaluate_batch(print_sents = True)

Summarizing Batch...
Summarizing Batch...
INFO:tensorflow:Finished reading dataset in single_pass mode.


In [87]:
scores

{'rouge-1': {'f': 0.11285714137734695, 'p': 0.1125, 'r': 0.11666666666666665},
 'rouge-2': {'f': 0.036666666176666675,
  'p': 0.041666666666666664,
  'r': 0.03333333333333333},
 'rouge-l': {'f': 0.13047618906195013,
  'p': 0.15833333333333333,
  'r': 0.11666666666666665}}

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000


In [88]:
ref_sents

["insert appropriate seller's payment",
 'call sender',
 'save power point',
 'examine this',
 'send findings to sender',
 'change calendar',
 'respond to sender with comments about it',
 'complete document and return to sender',
 'give patti ondetails',
 'communicate with mark holsworth',
 'reply to sender about ocr program',
 'inform sender of needs to load the gtcs',
 'review suggested mark up',
 'give number of copies to sender',
 'verify pg e topock curves are flat to socal',
 'contact sender with further comments and to coordinate execution',
 'remind sender when in banff louise',
 'look at book review',
 'return toaster to sender',
 'forward it to jon']

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000


In [86]:
decoded_sents

['xxx',
 'xxx',
 'xxx',
 'kate you',
 'please find out',
 'can on calendar',
 'please asap asap asap',
 'if send send',
 'please ondetails ondetails ondetails',
 'please holsworth holsworth holsworth',
 'xxx',
 'xxx',
 'xxx',
 'xxx',
 'xxx',
 'xxx',
 'xxx',
 'please book review',
 'would toaster toaster toaster',
 'could to jon jon']

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 32000


In [5]:
from setuptools import setup, find_packages


In [6]:
find_packages()

['data_util']

In [8]:
d

AttributeError: module 'os' has no attribute 'list_dir'